In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def month_year_iter(start_month, start_year, end_month, end_year):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range(ym_start, ym_end):
        y, m = divmod(ym, 12)
        yield y, m+1

In [3]:
rCEBE=r"C:\Users\jcleiva\Documents\Reportes Base\{}\Costo de Ventas\{}. Costo de Ventas Industria CEBE.xlsx"
rDesp=r"C:\Users\jcleiva\Documents\Reportes Base\{}\Despachos\{}. Despachos {}Q.xlsx"
rDest=r"C:\Users\jcleiva\Documents\Reportes Base\{}\Diferencia SAP-SINCO\{}. Diferencia SAP-SINCO.xlsx"
rM=r"C:\Users\jcleiva\Documents\Reportes Base\Maestras\{}"

In [4]:
rcCols=["Documento comercial","Centro","Número de cuenta","Denominación","Centro de beneficio",
        "Material","En moneda local centro de beneficio","Cantidad"]
rcConv={"Documento comercial":str,"Número de cuenta":str,"Centro":str,"Centro de beneficio":str,"Material":str}

In [5]:
rdCols=["Entrega","N° Ped EDI","Destinatario de mercancías","    # PLU","Material","Número de material","Cantidad entrega",
       "Peso neto","SBL","Fe.SM real","V.Neto"]
rdConv={"Entrega":str,"N° Ped EDI":str,"Destinatario de mercancías":str,"    # PLU":str,"Número de material":str,"SBL":str,
       "Material":str}

In [6]:
mesA=9
yearA=2023

mesI=11
yearI=2023

In [7]:
for tiempo in month_year_iter(mesA,yearA,mesI,yearI):
    dfCV=pd.read_excel(rCEBE.format(tiempo[0],tiempo[1]),usecols=rcCols,converters=rcConv)
    dfDesp=pd.read_excel(rDesp.format(tiempo[0],tiempo[1],1),usecols=rdCols,converters=rdConv)
    if datetime(tiempo[0],tiempo[1],16) <= datetime.now():
        dfDesp=pd.concat([dfDesp,pd.read_excel(rDesp.format(tiempo[0],tiempo[1],2),converters=rdConv,usecols=rdCols)])
    
    dfDesp=dfDesp.rename(columns={"    # PLU":"PLU",
                              "Entrega":"Documento comercial",
                              "Número de material":"Desc. Material",
                             "Cantidad entrega":"Cantidad entrega (Sinco)",
                             "Peso neto":"Cantidad entrega Kg (Sinco)",
                             "V.Neto":"Costo (Sinco)",
                             "SBL":"Sublinea"})

    dfDesp=dfDesp.groupby(['Fe.SM real', 'Destinatario de mercancías', 
                    'N° Ped EDI','Documento comercial', 
                    'PLU', "Material",'Desc. Material','Sublinea']).sum().reset_index()

    dfCV=dfCV[dfCV["Número de cuenta"]=="612014014"]

    dfCV=dfCV.groupby(['Centro de beneficio', 'Número de cuenta', 'Denominación',
                       'Material', 'Centro','Documento comercial'],dropna=False).sum().reset_index()

    dfCV=dfCV.rename(columns={"Cantidad":"Cantidad (SAP)",
                             "En moneda local centro de beneficio":"Costo (SAP)"})

    if dfDesp.shape[0] != dfDesp.merge(dfCV, on=["Documento comercial","Material"],how="left").shape[0]:
        raise Exception("Inserta filas")

    dfDesp=dfDesp.merge(dfCV, on=["Documento comercial","Material"],how="left")
    
    dM=pd.read_excel(rM.format("Centros.xlsx"),usecols=["Centro","Descripción Centro"],converters={"Centro":str})
    dfDesp=dfDesp.merge(dM,on=["Centro"],how="left")
    
    dM=pd.read_excel(rM.format("Maestro CEBE Industria.xlsx"),usecols=["Centro de beneficio","Denominación CEBE"],converters={"Centro de beneficio":str})
    dfDesp=dfDesp.merge(dM,on=["Centro de beneficio"],how="left")
    
    dfDesp["Sublinea"]=dfDesp["Sublinea"].str.lstrip("0")
    
    dM=pd.read_excel(rM.format("Sublinea2.xlsx"),usecols=["Sub","Sublinea","GEN"],converters={"Sub":str})
    dM=dM.rename(columns={"Sub":"Sublinea","Sublinea":"Desc. Sublinea"})
    dfDesp=dfDesp.merge(dM,on=["Sublinea"],how="left")
    
    dfDesp.to_excel(rDest.format(tiempo[0],tiempo[1]),index=None)
    print(tiempo)

(2023, 9)
(2023, 10)


In [3]:
"{:03d}.{}".format(12,2023)

'012.2023'